In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for file_or_dir in os.listdir('/kaggle/input/plant-leaf-dataset/Plant leaf dataset'):
    print(file_or_dir)

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

valid
test
train


In [3]:
# Imports

import os
import glob
import numpy as np
import pandas as pd


from pprint import pprint

import tensorflow as tf

from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras import Model, layers, Sequential, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint

import matplotlib.pyplot as plt

In [4]:
print("GPUs Available: ", tf.config.experimental.list_physical_devices('GPU'))

GPUs Available:  [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [5]:
train_path = "/kaggle/input/plant-leaf-dataset/Plant leaf dataset/train"
valid_path = "/kaggle/input/plant-leaf-dataset/Plant leaf dataset/valid"
test_path = "/kaggle/input/plant-leaf-dataset/Plant leaf dataset/test"

In [6]:
training_images = []

for root, dirs, files in os.walk(train_path):
    for file in files:
        training_images.append(os.path.join(root, file))
                               
validation_images = []

for root, dirs, files in os.walk(valid_path):
    for file in files:
        validation_images.append(os.path.join(root, file))

In [7]:
print("Training:")
print("Training Path:" + train_path)
print("Training Classes:" + str(len(os.listdir(train_path))))
print("Training Images:" + str(len(training_images)))

print("\n")

print("Validation:")
print("Validation Path:" + valid_path)
print("Validation Classes:" + str(len(os.listdir(valid_path))))
print("Validation Images:" + str(len(validation_images)))

print("\n")

print("Testing:")
print("Testing Path:" + test_path)
print("Testing Images:" + str(len(os.listdir(test_path))))

Training:
Training Path:/kaggle/input/plant-leaf-dataset/Plant leaf dataset/train
Training Classes:41
Training Images:65447


Validation:
Validation Path:/kaggle/input/plant-leaf-dataset/Plant leaf dataset/valid
Validation Classes:41
Validation Images:16724


Testing:
Testing Path:/kaggle/input/plant-leaf-dataset/Plant leaf dataset/test
Testing Images:49


In [8]:
train_datagen = ImageDataGenerator(
    rescale=1/255,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    shear_range=0.2,
    fill_mode='nearest'
)

validation_datagen = ImageDataGenerator(rescale=1/255)

test_datagen = ImageDataGenerator(rescale=1/255)

In [9]:
batch_size = 128

In [10]:
train_generator = train_datagen.flow_from_directory(
    train_path,
    batch_size = batch_size,
    class_mode = 'categorical',
    target_size = (224, 224),
    color_mode="rgb",
    shuffle=True
)

validation_generator =  validation_datagen.flow_from_directory(
    valid_path,
    batch_size  = batch_size,
    class_mode  = 'categorical',
    target_size = (224, 224),
    color_mode="rgb",
    shuffle=True
)

test_generator = test_datagen.flow_from_directory(
    test_path,
    batch_size  = 1,
    class_mode  = None,
    target_size = (224, 224),
    color_mode="rgb",
    shuffle=False
)

Found 65447 images belonging to 41 classes.
Found 16724 images belonging to 41 classes.
Found 0 images belonging to 0 classes.


In [11]:
class_dict = train_generator.class_indices
pprint(class_dict)

{'Apple___Apple_scab': 0,
 'Apple___Black_rot': 1,
 'Apple___Cedar_apple_rust': 2,
 'Apple___healthy': 3,
 'Cherry_(including_sour)___Powdery_mildew': 4,
 'Cherry_(including_sour)___healthy': 5,
 'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot': 6,
 'Corn_(maize)___Common_rust_': 7,
 'Corn_(maize)___Northern_Leaf_Blight': 8,
 'Corn_(maize)___healthy': 9,
 'Grape___Black_rot': 10,
 'Grape___Esca_(Black_Measles)': 11,
 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)': 12,
 'Grape___healthy': 13,
 'Orange___Haunglongbing_(Citrus_greening)': 14,
 'Peach___Bacterial_spot': 15,
 'Peach___healthy': 16,
 'Pepper,_bell___Bacterial_spot': 17,
 'Pepper,_bell___healthy': 18,
 'Potato___Early_blight': 19,
 'Potato___Late_blight': 20,
 'Potato___healthy': 21,
 'Rice__Bacterial leaf blight': 22,
 'Rice__Brown spot': 23,
 'Rice__Leaf smut': 24,
 'Soybean___healthy': 25,
 'Strawberry___Leaf_scorch': 26,
 'Strawberry___healthy': 27,
 'Tomato___Bacterial_spot': 28,
 'Tomato___Early_blight': 29,
 'Tomato_

In [12]:
class_list = list(class_dict.keys())
pprint(class_list)

['Apple___Apple_scab',
 'Apple___Black_rot',
 'Apple___Cedar_apple_rust',
 'Apple___healthy',
 'Cherry_(including_sour)___Powdery_mildew',
 'Cherry_(including_sour)___healthy',
 'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot',
 'Corn_(maize)___Common_rust_',
 'Corn_(maize)___Northern_Leaf_Blight',
 'Corn_(maize)___healthy',
 'Grape___Black_rot',
 'Grape___Esca_(Black_Measles)',
 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)',
 'Grape___healthy',
 'Orange___Haunglongbing_(Citrus_greening)',
 'Peach___Bacterial_spot',
 'Peach___healthy',
 'Pepper,_bell___Bacterial_spot',
 'Pepper,_bell___healthy',
 'Potato___Early_blight',
 'Potato___Late_blight',
 'Potato___healthy',
 'Rice__Bacterial leaf blight',
 'Rice__Brown spot',
 'Rice__Leaf smut',
 'Soybean___healthy',
 'Strawberry___Leaf_scorch',
 'Strawberry___healthy',
 'Tomato___Bacterial_spot',
 'Tomato___Early_blight',
 'Tomato___Late_blight',
 'Tomato___Leaf_Mold',
 'Tomato___Septoria_leaf_spot',
 'Tomato___Spider_mites Two-spotted_spi

In [13]:
# Creating Instence of pre-trained model from Keras Application
inception_model = InceptionV3(input_shape= (224, 224, 3),
                                include_top = False,
                                weights = 'imagenet')

87910968/87910968 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [14]:
train_num = train_generator.samples
valid_num = validation_generator.samples

In [15]:
# Freezing all Layers of Inception V3 Model
for layer in inception_model.layers:
    layer.trainable = False

In [16]:
inception_model.summary()

Model: "inception_v3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 111, 111,  │        864 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 111, 111,  │         96 │ conv2d[0][0]      │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 111, 111,  │          0 │ batch_normalizat… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 109, 109,  │      9,216 │ activation[0][0]  │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 109, 109,  │         96 │ conv2d_1[0][0]    │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_1        │ (None, 109, 109,  │          0 │ batch_normalizat… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 109, 109,  │     18,432 │ activation_1[0][… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 109, 109,  │        192 │ conv2d_2[0][0]    │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_2        │ (None, 109, 109,  │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 54, 54,    │          0 │ activation_2[0][… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 54, 54,    │      5,120 │ max_pooling2d[0]… │
│                     │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 54, 54,    │        240 │ conv2d_3[0][0]    │
│ (BatchNormalizatio… │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_3        │ (None, 54, 54,    │          0 │ batch_normalizat… │
│ (Activation)        │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 52, 52,    │    138,240 │ activation_3[0][… │
│                     │ 192)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 52, 52,    │        576 │ conv2d_4[0][0]    │
│ (BatchNormalizatio… │ 192)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_4        │ (None, 52, 52,    │          0 │ batch_normalizat

 Total params: 21,802,784 (83.17 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 21,802,784 (83.17 MB)

In [17]:
last_layer = inception_model.get_layer('mixed9')
last_output = last_layer.output
print('Last Layer Output Shape:', last_output.shape)

Last Layer Output Shape: (None, 5, 5, 2048)


In [18]:
# Adding our own layers at the end of Inception Network

# Flatten the output layer to 1 dimension
x = layers.Flatten()(last_output)

# Add a fully connected layer with 1024 hidden units and ReLU activation
x = layers.Dense(2048, activation='relu')(x)

# Add a fully connected layer with 1024 hidden units and ReLU activation
x = layers.Dense(1024, activation='relu')(x)

# Add a dropout rate of 0.2
x = layers.Dropout(0.2)(x)

# Add a final sigmoid layer for classification
x = layers.Dense(len(class_dict), activation='softmax')(x)

model = Model(inception_model.input, x)

In [19]:
model.compile(optimizer=optimizers.Adam(learning_rate=0.001),
              loss = 'categorical_crossentropy',
              metrics = ['accuracy'])

In [20]:
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 111, 111,  │        864 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 111, 111,  │         96 │ conv2d[0][0]      │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 111, 111,  │          0 │ batch_normalizat… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 109, 109,  │      9,216 │ activation[0][0]  │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 109, 109,  │         96 │ conv2d_1[0][0]    │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_1        │ (None, 109, 109,  │          0 │ batch_normalizat… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 109, 109,  │     18,432 │ activation_1[0][… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 109, 109,  │        192 │ conv2d_2[0][0]    │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_2        │ (None, 109, 109,  │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 54, 54,    │          0 │ activation_2[0][… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 54, 54,    │      5,120 │ max_pooling2d[0]… │
│                     │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 54, 54,    │        240 │ conv2d_3[0][0]    │
│ (BatchNormalizatio… │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_3        │ (None, 54, 54,    │          0 │ batch_normalizat… │
│ (Activation)        │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 52, 52,    │    138,240 │ activation_3[0][… │
│                     │ 192)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 52, 52,    │        576 │ conv2d_4[0][0]    │
│ (BatchNormalizatio… │ 192)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_4        │ (None, 52, 52,    │          0 │ batch_normalizat

 Total params: 122,722,569 (468.15 MB)

 Trainable params: 106,999,849 (408.17 MB)

 Non-trainable params: 15,722,720 (59.98 MB)

In [22]:
try:
  history = model.fit(train_generator,steps_per_epoch=train_num//batch_size,
                    validation_data = validation_generator,validation_steps=valid_num//batch_size,
                    epochs = 15,)
except Exception as e:
  print("An error occurred during training:", e)

Epoch 1/15
511/511 ━━━━━━━━━━━━━━━━━━━━ 1114s 2s/step - accuracy: 0.9194 - loss: 0.2463 - val_accuracy: 0.9366 - val_loss: 0.1861
Epoch 2/15
511/511 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9453 - loss: 0.1510 - val_accuracy: 0.9286 - val_loss: 0.2049
Epoch 3/15
511/511 ━━━━━━━━━━━━━━━━━━━━ 1107s 2s/step - accuracy: 0.9374 - loss: 0.1886 - val_accuracy: 0.9505 - val_loss: 0.1470
Epoch 4/15
511/511 ━━━━━━━━━━━━━━━━━━━━ 0s 236us/step - accuracy: 0.9531 - loss: 0.1300 - val_accuracy: 0.9524 - val_loss: 0.1208
Epoch 5/15
511/511 ━━━━━━━━━━━━━━━━━━━━ 1108s 2s/step - accuracy: 0.9450 - loss: 0.1725 - val_accuracy: 0.9521 - val_loss: 0.1466
Epoch 6/15
511/511 ━━━━━━━━━━━━━━━━━━━━ 0s 229us/step - accuracy: 0.9453 - loss: 0.1796 - val_accuracy: 0.9524 - val_loss: 0.0855
Epoch 7/15
511/511 ━━━━━━━━━━━━━━━━━━━━ 1104s 2s/step - accuracy: 0.9534 - loss: 0.1450 - val_accuracy: 0.9537 - val_loss: 0.1390
Epoch 8/15
511/511 ━━━━━━━━━━━━━━━━━━━━ 0s 228us/step - accuracy: 0.9688 - loss: 0.0912 - va

In [23]:
model.save("InceptionV3_15Epoch.h5")

In [ ]:
test = "/kaggle/input/plant-leaf-dataset/Plant leaf dataset/test/CornCommonRust1.JPG"

In [ ]:
import cv2
import numpy as np

# Load the image
img = cv2.imread(test)

# Preprocess the image
img = cv2.resize(img, (224, 224))
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img = img / 255.0

# Add batch dimension
img = np.expand_dims(img, axis=0)

# Predict
y_pred = model.predict(img)
y_pred_bool = np.argmax(y_pred, axis=1)

In [136]:
y_pred_bool

array([7])

In [ ]:
Inception_V3 = tf.keras.models.load_model("InceptionV3_trained\InceptionV3_5Epoch.h5")